This notebook compares the patches generated by MapReader with the StopGB dataset

* Which Stops from StopGB are or are not within a MapReader Patch?
  * If they are which patch ID?

For each Stop within StopsGB:
  * Is it within a MapReader patch? If so which patch?
  * If it is not, is it with a XXX meter buffer of the patches? If so which is the nearest patch centroid?


Also:
* Can this be used to identify errors in either the StopsGB or MapReader Patch data?


In [131]:
# ! pip install gspread
# ! pip install df2gspread

# ! pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
# 'conda install pygeos' or 
# ! pip install pygeos
# ! python --version

In [132]:
import pandas as pd
import geopandas
from icecream import ic
import pygeos


In [133]:

# # Taken from https://towardsdatascience.com/how-to-import-google-sheets-data-into-a-pandas-dataframe-using-googles-api-v4-2020-f50e84ea4530

# import pickle
# import os.path
# from google_auth_oauthlib.flow import InstalledAppFlow
# from google.auth.transport.requests import Request
# from googleapiclient.discovery import build


# def gsheet_api_check(SCOPES):
#     creds = None
#     if os.path.exists('token.pickle'):
#         with open('token.pickle', 'rb') as token:
#             creds = pickle.load(token)
            
#     if not creds or not creds.valid:
#         if creds and creds.expired and creds.refresh_token:
#             creds.refresh(Request())
#         else:
#             flow = InstalledAppFlow.from_client_secrets_file(
#                 'credentials.json', SCOPES)
#             creds = flow.run_local_server(port=0)

#         with open('token.pickle', 'wb') as token:
#             pickle.dump(creds, token)

#     return creds


# def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
#     creds = gsheet_api_check(SCOPES)
#     service = build('sheets', 'v4', credentials=creds)
#     sheet = service.spreadsheets()
#     result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=DATA_TO_PULL).execute()
#     values = result.get('values', [])
    
#     if not values:
#         print('No data found.')
#     else:
#         rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
#                                   range=DATA_TO_PULL).execute()
#         data = rows.get('values')
#         print("COMPLETE: Data copied")
#         return data


# Some handly functions for reading and writing to Google Spreadsheets

In [134]:
"""
Some handly functions to wrap up reading and writing to Google Spreadsheets

See https://docs.gspread.org/en/latest/index.html for details
"""
import os.path
import gspread


def open_gspreadsheet(gss_url, credentials_fpath=None, authorized_user_fpath=None):
    ipynb_path = os.path.dirname(os.path.realpath("__file__"))

    if not credentials_fpath:
        credentials_fpath = os.path.join(ipynb_path, "credentials.json")

    if not authorized_user_fpath:
        authorized_user_fpath = os.path.join(ipynb_path, "authorized_user.json")

    gc = gspread.oauth(
        credentials_filename=credentials_fpath,
        authorized_user_filename=authorized_user_fpath
    )

    gss = gc.open_by_url(gss_url)
    return gss

def get_sheet_as_dataframe(gss, sheet_name):
    ic([sheet.title for sheet in gss.worksheets()])
    sheet = gss.worksheet(sheet_name)
    dataframe = pd.DataFrame(sheet.get_all_records())
    return dataframe

def write_dataframe_to_sheet(dataframe, gss, sheet_name, overwrite_existing_sheet=False):
    """   
    Behaviour in different senarios: 
        in==T,  ow==T => Del, then create new
        in==T,  ow==F => Raise exception
        in==F,  ow==T => create new
        in==F,  ow==F => create new
    """
    if (sheet_name in [sheet.title for sheet in gss.worksheets()]):
        if overwrite_existing_sheet:
            old_sheet = gss.worksheet(sheet_name)
            gss.del_worksheet(old_sheet)
        else:
            raise ValueError(f"Cannot overwrite existing sheet '{sheet_name}'. Please use a"
                              " different `sheet_name` or select `overwrite_existing_sheet=True`")

    # Now create a new sheet
    nrows, ncols = dataframe.shape
    new_sheet = gss.add_worksheet(sheet_name, rows=nrows, cols=ncols)
    # And write out the data frame
    new_sheet.update([dataframe.columns.values.tolist()] + dataframe.values.tolist())


# Global Variables

In [135]:
"""
Global variables
"""
# Testing value:
# stopsgb_gss_url = "https://docs.google.com/spreadsheets/d/1nJpaT_e7dpDkqFLV2nLsnuGHngMKsD-t7vjtepVbfrs/edit#gid=879256170"
# input_sheet_name = "StopsGB_andy_org"
# "Real" value:
stopsgb_gss_url = "https://docs.google.com/spreadsheets/d/1_9-dIJXrLAVEgDKYJe6YQNCQu5rP7eOwPGW6zxydBw4/edit#gid=879256170"
input_sheet_name = "StopsGB"

output_sheet_name = "StopsGB_with_patch_info"

ipynb_path = os.path.dirname(os.path.realpath("__file__"))
# "Real" value:
stopsgb_gpkg_path = os.path.join(ipynb_path, "stops_gb.gpkg")
stopsgb_layer_name = "joined_stops_gb"
# Temp Stops Exctract for testing
# stopsgb_gpkg_path = "/Users/a.smith/Documents.OLD/Living-with-machines/mapreader_display_data/pred_squares_extract.gpkg"
# stopsgb_layer_name = "stops_gb_extract"

ic(stopsgb_gpkg_path)

# Testing value:
# patches_gpkg_path = "/Users/a.smith/Documents.OLD/Living-with-machines/mapreader_display_data/pred_squares_extract.gpkg"
# patches_layer_name = "patches_0103_extract"
# "Real" value:
patches_gpkg_path = "/Users/a.smith/Documents.OLD/Living-with-machines/mapreader_display_data/pred_squares2.gpkg"
patches_layer_name = "patches_0103_all_v003"

patch_centroids_path = "/Users/a.smith/Documents.OLD/Living-with-machines/mapreader_display_data/pred.gpkg"
patch_centroids_layer_name = "pred_0103_all_v003"

maxium_distance = 2000 # meters


ic| stopsgb_gpkg_path: '/Users/a.smith/Library/CloudStorage/OneDrive-TheAlanTuringInstitute/Documents/Living-with-machines/code/MapReader/examples/postproc/stops_gb.gpkg'


# Read the StopsGB data from a Google Speadsheet into a GeoPKG

In [136]:
from pandas.api.types import is_numeric_dtype

# Open the Google Spreadsheet
gss = open_gspreadsheet(gss_url=stopsgb_gss_url)
df = get_sheet_as_dataframe(gss, input_sheet_name)

# Filter out "unknown" values.
ic(len(df))
filtered_df = df.loc[(df.selected_entity_longitude != "unknown") & (df.selected_entity_latitude != "unknown")]
ic(len(filtered_df))

# Create GeoDF from `selected_entity_latitude` and `selected_entity_longitude` values.
source_crs = "EPSG:4326"
stops_gb_gdf = geopandas.GeoDataFrame(
    filtered_df, geometry=geopandas.points_from_xy(filtered_df.selected_entity_longitude, filtered_df.selected_entity_latitude, crs=source_crs))

# Write out to GeoPKG
stops_gb_gdf.to_file(stopsgb_gpkg_path, driver="GPKG", layer=stopsgb_layer_name)
ic(len(stops_gb_gdf))


ic| [sheet.title for sheet in gss.worksheets()]: ['StopsGB']
ic| len(df): 12676
ic| len(filtered_df): 12142
/usr/local/anaconda3/envs/geo_env/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
ic| len(stops_gb_gdf): 12142


12142

# Load the patches GeoPKGs

In [137]:
# Load patches polygons
from shapely.geometry.linestring import LineString
from pandas import to_numeric, NamedAgg

patches_gdf = geopandas.read_file(patches_gpkg_path, layer=patches_layer_name)
patch_centroids_gdf = geopandas.read_file(patch_centroids_path, layer=patch_centroids_layer_name)
# patch_centroids_gdf.set_crs("EPSG:4326")
stops_gb_gdf = geopandas.read_file(stopsgb_gpkg_path, layer=stopsgb_layer_name)

# Calculate the number of Patches containing each Stop 

In [138]:
"""
Calculate the number of Patches containing each Stop (0,1, or rarely >1) 
"""
# Convert to common CRS
stops_gb_gdf = stops_gb_gdf.to_crs("EPSG:3857")
# Join the stops and the patches
stops_within_patch_gdf = stops_gb_gdf.sjoin(patches_gdf, how="left", predicate="within")

# Drop everything except the station ID and patch ID
stops_within_patch_gdf = stops_within_patch_gdf[[
    "StationId",
    "image_id",
]]

# Filter out any cases where there isn't a matching patch ID
# NOTE: Turns out that this isn't necessary
# stops_within_patch_gdf = stops_within_patch_gdf.loc[lambda stops_within_patch_gdf: ~stops_within_patch_gdf["image_id"].isna(), :]
# ic(stops_within_patch_gdf)

# Group by to get cases where there is more than one patch per stop.
groupby_station_id_df = stops_within_patch_gdf.groupby(by="StationId")
patch_count_df = groupby_station_id_df.agg(patch_count=NamedAgg(column="image_id", aggfunc="count"))
ic(len(patch_count_df))
ic(patch_count_df.head())

# Now join "patch_count" column back into StopsGB
stops_within_patch_gdf = stops_gb_gdf.join(patch_count_df, how="left", on="StationId")


ic| len(patch_count_df): 12142
ic| patch_count_df.head():            patch_count
                           StationId             
                           1                    1
                           2                    1
                           3                    1
                           4                    1
                           5                    0


# Calculate the closest patches centroid.

In [139]:
"""
Calculate the closest patches centroid.
Add columns for:
* PatchID
* Distance
"""
import numpy as np
# Convert to a common CRS
patch_centroids_gdf = patch_centroids_gdf.to_crs("EPSG:3857")

nearest_gdf = stops_within_patch_gdf.sjoin_nearest(
    patch_centroids_gdf,
    how="left",
    max_distance=maxium_distance,
    rsuffix="right_",
    distance_col="distance_to_patch_centroid"
)

nearest_gdf = nearest_gdf.fillna("")
# ic(nearest_gdf.head(10))
ic(nearest_gdf['distance_to_patch_centroid'].describe())

ic| nearest_gdf['distance_to_patch_centroid'].describe(): count     12323
                                                          unique     8783
                                                          top            
                                                          freq        898
                                                          Name: distance_to_patch_centroid, dtype: object


count     12323
unique     8783
top            
freq        898
Name: distance_to_patch_centroid, dtype: object

# Write results back to GSS

In [140]:
"""
Now write results back to GSS
"""
# Exclude geometry column:
tempdf = nearest_gdf.loc[:, nearest_gdf.columns!='geometry']
write_dataframe_to_sheet(tempdf, gss, output_sheet_name, overwrite_existing_sheet=True)
